In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mlaut.data import Data
from mlaut.estimators.estimators import instantiate_default_estimators
from mlaut.experiments import Orchestrator
from mlaut.analyze_results import AnalyseResults
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mlaut.analyze_results.scores import ScoreAccuracy
import pandas as pd

MLAUT is a modelling and workflow toolbox in python, written with the aim of simplifying large scale benchmarking of machine learning strategies, e.g., validation, evaluation and comparison with respect to predictive/task-specific performance or runtime.

In this basis use case example we will show the most simple MLAUT workflow.

For the purposes of this demonstration we will assume that the user has already stored the data needed for the experiments in a HDF5 database. Saving the data is not part of the core MLAUT workflow and is therefore omitted for the purposes of this this demonstration. 

Please refer to the advanced use case demos for examples how this can be done.

the diagram below sketches the typical MLAUT workflow.

<img src="img/workflow.png?2">

### Step 1: Database

The code below provides hooks to the input and output database objects

In [2]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='a')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

`input_io`: hook to the input HDF5 database file <br>
`out_io`:  hook to the output HDF5 database file

### Step 2: Split datasets

After the hooks are created we can proceed to splitting the data in test and training. 

Unless otherwise specified we use $\dfrac{2}{3}$ of the data for training and $\dfrac{1}{3}$ for testing. We do not change or move the original data in this process. Instead we store the train/test indices in a separate HDF5 database.


In [3]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path, verbose=False)

`dts_names_list`: names of the datasets saved inside the HDF5 file <br>
`dts_names_list_full_path`: full path to the datasets inside the HDF5 database <br>
`split_dts_list`: path to the train/test indices of the split datasets

### Step 3: Define the estimators

For the puposes of the basic demo we show how the standard set of estimaots that come with MLAUT can be used for running the experiments. 

In the code example below we enumerate by name the estimators that we wish to use in the study. This will provide instances of MLAUT estimators with the built in defaults.

For more advanced used cases pleaes refer to the Advanced Usage - Example 1 and Example 2. The user can easily change the hyper paramemeter defaults or define a completely new estimator object .

In [4]:
est = ['RandomForestClassifier','BaggingClassifier','GradientBoostingClassifier','SVC','GaussianNaiveBayes','BernoulliNaiveBayes','NeuralNetworkDeepClassifier','PassiveAggressiveClassifier','BaselineClassifier']
estimators = instantiate_default_estimators(estimators=est)

`estimators`: array of MLAUT estimators

### Step 4: Run the experiments

The final step is to run the experiments by invoking the `run()` method.

This step could take a substantial amount of time depending on the number and size of datasets and the number of estimators that we wish to train.

All trained estimators are saved on the HDD.

In [7]:
orchest = Orchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io, dts_names=dts_names_list,
                 original_datasets_group_h5_path='delgado_datasets/')
orchest.run(modelling_strategies=estimators, verbose=True)

2018-10-04 19:53:42,798 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,798 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,811 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,811 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,862 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,862 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on abalone. Skipping it.
2018-10-04 19:53:42,868 [MainThread  ] [INFO ]  Estimator SVC already trained on abalone. Skipping it.
2018-10-04 19:53:42,868 [MainThread  ] [INFO ]  Estimator SVC already trained on abalone. Skipping it.
2018-10-04 19:53:42,874 [MainThread  ] [INFO ]  Estimator Gaussi

2018-10-04 19:54:25,532 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on adult. Skipping it.
2018-10-04 19:54:25,535 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on adult. Skipping it.
2018-10-04 19:54:25,535 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on adult. Skipping it.
2018-10-04 19:54:25,538 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on adult. Skipping it.
2018-10-04 19:54:25,538 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on adult. Skipping it.
2018-10-04 19:54:25,566 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on annealing. Skipping it.
2018-10-04 19:54:25,566 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on annealing. Skipping it.
2018-10-04 19:54:25,574 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on annealing. Skipping it.
2018-10-04 19:54:25,574 [M

2018-10-04 19:55:00,197 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on balance_scale. Skipping it.
2018-10-04 19:55:00,197 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on balance_scale. Skipping it.
2018-10-04 19:55:00,201 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on balance_scale. Skipping it.
2018-10-04 19:55:00,201 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on balance_scale. Skipping it.
2018-10-04 19:55:11,203 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on balance_scale. Skipping it.
2018-10-04 19:55:11,203 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on balance_scale. Skipping it.
2018-10-04 19:55:11,205 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on balance_scale. Skipping it.
2018-10-04 19:55:11,205 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained o

2018-10-04 19:55:44,715 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,718 [MainThread  ] [INFO ]  Estimator SVC already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,718 [MainThread  ] [INFO ]  Estimator SVC already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,723 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,723 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,727 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on breast_cancer. Skipping it.
2018-10-04 19:55:44,727 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on breast_cancer. Skipping it.
2018-10-04 19:55:56,610 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on breast_cancer. Skipping it.
2018-10-04 19:55:56

2018-10-04 19:56:32,619 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on breast_cancer_wisc_prog. Skipping it.
2018-10-04 19:56:32,619 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on breast_cancer_wisc_prog. Skipping it.
2018-10-04 19:56:32,621 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on breast_cancer_wisc_prog. Skipping it.
2018-10-04 19:56:32,621 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on breast_cancer_wisc_prog. Skipping it.
2018-10-04 19:56:32,643 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on breast_tissue. Skipping it.
2018-10-04 19:56:32,643 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on breast_tissue. Skipping it.
2018-10-04 19:56:32,650 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on breast_tissue. Skipping it.
2018-10-04 19:56:32,650 [MainThread  ] [INFO ]  Estimator BaggingCl

...

2018-10-04 20:14:08,858 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,858 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,861 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,861 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,863 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,863 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on pittsburg_bridges_SPAN. Skipping it.
2018-10-04 20:14:08,892 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on pittsburg_bridges_TYPE. Skipping it.
2018-10-04 20:14:08,892 [Main

2018-10-04 20:15:10,840 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,988 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,988 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,992 [MainThread  ] [INFO ]  Estimator SVC already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,992 [MainThread  ] [INFO ]  Estimator SVC already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,995 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on plant_margin. Skipping it.
2018-10-04 20:15:10,995 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on plant_margin. Skipping it.
2018-10-04 20:15:11,001 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on plant_margin. Skipping it.
2018-10-04 20:15:11,001 [MainT

***************************
********  EXITING  ********
***************************


SystemExit: 

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


One of the key feautres of the package is to allow for the experiment to resume in case of a crash or interruption. If this happens, the user would simply need to re-run the code above. Unless the `override_saved_models=True` flag was set the orchestrator will skip all estimators that were trained sucessfully. This would allow the user to continue from the point where the experiments were stopped.

### Step 5: Make predictions on the test sets

After the estimators are trained the user needs to use them in order to make predictions on the test sets which will be used subsequently for performing statistical tests.

The predictions of the estimators are saved in the input HDF5 database file a hook to which was created earlier.

Unless the `override=False` flag was set MLAUT will not override predictions that were previously stored in the database.

In [6]:
orchest.predict_all(trained_models_dir='data/trained_models', estimators=estimators, verbose=True)

2018-10-04 19:53:13,228 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,229 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,231 [MainThread  ] [INFO ]  Predictions for SVC on abalone already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,232 [MainThread  ] [INFO ]  Predictions for BaggingClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,234 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,237 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on abalone already exist in the database. S

2018-10-04 19:53:13,497 [MainThread  ] [INFO ]  Predictions for SVC on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,498 [MainThread  ] [INFO ]  Predictions for BaggingClassifier on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,500 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,502 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,507 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,510 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on arrhythmia already exist in the datab

2018-10-04 19:53:13,729 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on blood already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,731 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on blood already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,733 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on blood already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,736 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on blood already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,738 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on blood already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,773 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on breast_cancer already exist in the da

2018-10-04 19:53:13,958 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on breast_tissue already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,960 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on breast_tissue already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,963 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on breast_tissue already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,964 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on breast_tissue already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,966 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on breast_tissue already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:13,995 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier 

2018-10-04 19:53:14,176 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on chess_krvkp already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:14,177 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on chess_krvkp already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:14,179 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on chess_krvkp already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:14,181 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on chess_krvkp already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:14,214 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on congressional_voting already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:14,216 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on co

...

2018-10-04 19:53:18,090 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on statlog_german_credit already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,092 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on statlog_german_credit already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,095 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on statlog_german_credit already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,097 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on statlog_german_credit already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,137 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on statlog_heart already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,140 [MainThread  ] [INFO ]  Predictions

2018-10-04 19:53:18,351 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on statlog_vehicle already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,355 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on statlog_vehicle already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,362 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on statlog_vehicle already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,364 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on statlog_vehicle already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,389 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on steel_plates already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,391 [MainThread  ] [INFO ]  Predictions for RandomForestClassifi

2018-10-04 19:53:18,570 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on tic_tac_toe already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,572 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on tic_tac_toe already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,573 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on tic_tac_toe already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,594 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on titanic already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,596 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on titanic already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,598 [MainThread  ] [INFO ]  Predictions for SVC on titanic already exist in the database. Set 

2018-10-04 19:53:18,779 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on vertebral_column_3clases already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,781 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on vertebral_column_3clases already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,806 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on wall_following already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,808 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on wall_following already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,809 [MainThread  ] [INFO ]  Predictions for SVC on wall_following already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:18,812 [MainThread  ] [INFO ]  Predictions for BaggingClassifier on wa

2018-10-04 19:53:18,995 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on wine_quality_red already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:19,016 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on wine_quality_white already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:19,019 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on wine_quality_white already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:19,025 [MainThread  ] [INFO ]  Predictions for SVC on wine_quality_white already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:19,027 [MainThread  ] [INFO ]  Predictions for BaggingClassifier on wine_quality_white already exist in the database. Set override to True if you wish replace them.
2018-10-04 19:53:19,029 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier

### Step 6: Analyze the results

The last step in the pipeline is to analyze the results of the experiments.

The `AnalyseResults` class takes as inputs the two database files and the loss metric that will be used to compute the prediction errors.

In [ ]:
analyze = AnalyseResults(hdf5_output_io=out_io, 
                         hdf5_input_io=input_io,
                         input_h5_original_datasets_group='delgado_datasets/', 
                         output_h5_predictions_group='experiments/predictions/')
score_accuracy = ScoreAccuracy()


(errors_per_estimator, 
 errors_per_dataset_per_estimator, 
 errors_per_dataset_per_estimator_df) = analyze.prediction_errors(score_accuracy, estimators)


The `prediction_errors()` method retuns two sets of results: `errors_per_estimator` dictionary which is used subsequently in further statistical tests and `errors_per_dataset_per_estimator_df` which is a dataframe with the loss of each estimator on each dataset which can be examined directly by the user. 

Below we show the results of the various statistical tests that are supported by MLAUT

#### t-test

In [ ]:
t_test, t_test_df = analyze.t_test(errors_per_estimator)
t_test_df

#### sign test

In [ ]:
sign_test, sign_test_df = analyze.sign_test(errors_per_estimator)
sign_test_df

#### t-test with bonferroni correction

In [ ]:
t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(errors_per_estimator)
t_test_bonferroni_df

#### Wilcoxon test

In [ ]:
import warnings
warnings.filterwarnings('ignore')
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(errors_per_estimator)
wilcoxon_test_df

#### Friedman test

In [ ]:
friedman_test, friedman_test_df = analyze.friedman_test(errors_per_estimator)
friedman_test_df

#### Nemenyi test

In [ ]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test

In [ ]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test

In [ ]:
pd.set_option('display.max_rows', 5000)
errors_per_dataset_per_estimator_df